In [1]:
import os
import json
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.layers import InputLayer

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

keras의 backend에서 session을 하나 열어서 만드는 모델의 layers마다 numbering을 한다.<br>
초기화를 하고 싶을 때는, `tf.keras.backend.clear_session()`을 입력하면 된다.

In [2]:
model = Sequential()
# model.add(InputLayer(input_shape=(28, 28, 1))) # InputLayer를 사용할 때는 batch_size는 입력안함.
model.add(Flatten())
model.add(Dense(units=10))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))

# model을 build할 때 입력되는 input_shape=(batch_size, width, height, channel)
model.build(input_shape=(None, 28, 28, 1))
# model.build()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 7,872
Trainable params: 7,872
Non-trainable params: 0
_________________________________________________________________


In [3]:
class TestModel(Model):
  def __init__(self):
    super(TestModel, self).__init__()
    
    self.flatten = Flatten()
    self.d1 = Dense(units=10)
    self.d1_act = Activation('relu')
    self.d2 = Dense(units=2)
    self.d2_act = Activation('softmax')
    
  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d1_act(x)
    x = self.d2(x)
    x = self.d2_act(x)
    return x

model = TestModel()
if not model.built: # model이 build 되었는지 되지 않았는지 확인, 되지 않았다면 input_shape를 입력해서 build.
  model.build(input_shape=(None, 28, 28, 1))

model.summary()

Model: "test_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  7850      
_________________________________________________________________
activation_2 (Activation)    multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  22        
_________________________________________________________________
activation_3 (Activation)    multiple                  0         
Total params: 7,872
Trainable params: 7,872
Non-trainable params: 0
_________________________________________________________________


model을 build할 때, 데이터 세트를 입력하게 되면 자동적으로 데이터의 shape에 맞게 model이 build되기 때문에 굳이 build()를 쓸 필요는 없지만 learning 전에 분석을 하기 위해 사용할 수 있다. <br>그리고, model이 build가 된 후에는 각 layer에 runnable parameter가 세팅이 되었다는 것이기 때문에 layer의 정보나 처음으로 초기화가 되어 있는 weight, bias의 정보를 확인 할 수 있다.

In [4]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten, Dense, Activation

model = Sequential()
model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='valid', name='conv_1'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='conv_1_maxpool'))
model.add(Activation('relu', name='conv_1_act'))
model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='valid', name='conv_2'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='conv_2_maxpool'))
model.add(Activation('relu', name='conv_2_act'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu', name='dense_1'))
model.add(Dense(units=10, activation='softmax', name='dense_2'))

model.build(input_shape=(None, 28, 28, 1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 26, 26, 10)        100       
_________________________________________________________________
conv_1_maxpool (MaxPooling2D (None, 13, 13, 10)        0         
_________________________________________________________________
conv_1_act (Activation)      (None, 13, 13, 10)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 11, 11, 10)        910       
_________________________________________________________________
conv_2_maxpool (MaxPooling2D (None, 5, 5, 10)          0         
_________________________________________________________________
conv_2_act (Activation)      (None, 5, 5, 10)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 250)              

In [5]:
print(model.layers)

[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000025323DA0940>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000025323DA0370>, <tensorflow.python.keras.layers.core.Activation object at 0x000002534C0A8970>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000025323D4FAC0>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000025323D4FBE0>, <tensorflow.python.keras.layers.core.Activation object at 0x000002532164D9A0>, <tensorflow.python.keras.layers.core.Flatten object at 0x0000025323D4FB50>, <tensorflow.python.keras.layers.core.Dense object at 0x0000025323D83BB0>, <tensorflow.python.keras.layers.core.Dense object at 0x0000025323D83F10>]


In [6]:
print(model.layers[0].get_weights())

[array([[[[-0.16781989, -0.1248984 ,  0.05839747, -0.01067461,
          -0.23539779, -0.07740344, -0.16388512,  0.09321219,
          -0.14458762,  0.12068373]],

        [[-0.00652584, -0.06170386, -0.02808173, -0.17449963,
          -0.05268474,  0.07589334, -0.02689227,  0.05674851,
           0.21147752,  0.11131066]],

        [[ 0.18104142, -0.23223838,  0.24321344,  0.22069296,
           0.03963786, -0.21531537,  0.2131699 , -0.10960037,
          -0.03230315,  0.1656352 ]]],


       [[[-0.02999558,  0.19904459, -0.15691173,  0.1711562 ,
           0.00543922,  0.04869804,  0.0177286 ,  0.00505859,
          -0.23498006, -0.2089307 ]],

        [[-0.04659559, -0.06720361,  0.1253584 ,  0.01550266,
           0.19702679,  0.17184082, -0.1362399 , -0.17733252,
           0.00809345,  0.1288608 ]],

        [[ 0.24177748,  0.12073526, -0.00662011,  0.16366607,
          -0.02756369, -0.04697381, -0.107062  , -0.0269255 ,
          -0.09779359,  0.13863423]]],


       [[[-0.1217